<a href="https://colab.research.google.com/github/samuelacquatey/DS-study-notebooks/blob/main/ALX_Movie_Recommender_System_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [4]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357253 sha256=4dece63d55bc612fba1ed8338e66909d9ffd53acb7931a8e4d483065d24d4e4c
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [13]:
import pandas as pd
from surprise import Dataset, Reader, SVD

# Load training data
train = pd.read_csv('train.csv')  # replace 'train.csv' with your actual file path
ratings = train.drop(columns='timestamp')

# Prepare the data for surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings, reader)

# Initialize the SVD model
RANDOM_STATE = 42
svd = SVD(n_epochs=40, n_factors=1000, init_std_dev=0.005, random_state=RANDOM_STATE, verbose=True)

# Train the model on the full training set
trainset = data.build_full_trainset()
svd.fit(trainset)

# Load test data
test = pd.read_csv('test.csv')  # replace 'test.csv' with your actual file path

# Predict ratings for the test set
userId = test['userId'].values.tolist()
movieId = test['movieId'].values.tolist()

count = len(userId)
ratings = []
ids = []

for i in range(count):
    result = svd.predict(userId[i], movieId[i]).est
    ratings.append(result)
    ids.append(f"{userId[i]}_{movieId[i]}")

# Create the prediction DataFrame
pred_df = pd.DataFrame({
    'Id': ids,
    'rating': ratings
})

print(pred_df.head())


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
       Id    rating
0  1_2011  3.527600
1  1_4144  4.102319
2  1_5767  3.501296
3  1_6711  3.738204
4  1_7318  3.421147


In [12]:
test = pd.read_csv('test.csv')
print(test.shape)
print(test.head())


(5000019, 2)
   userId  movieId
0       1     2011
1       1     4144
2       1     5767
3       1     6711
4       1     7318


In [11]:
print(pred_df.shape)
pred_df.head()

(978757, 2)


,Id,rating
0,1_2011.0,3.527600
1,1_4144.0,4.102319
2,1_5767.0,3.501296
3,1_6711.0,3.738204
4,1_7318.0,3.421147


In [ ]:
preddiction

In [14]:
pred_df.to_csv('predictions.csv', index=False)

from google.colab import files
files.download('predictions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# Import our regular old heroes
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficiency.
import matplotlib.pyplot as plt
import seaborn as sns

#for standardizing some of my data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convenient item retrieval during iteration
import heapq # <-- Efficient sorting of large lists

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load datasets
train_df = pd.read_csv('train.csv')
genome_tags_df = pd.read_csv('genome_tags.csv')
genome_scores_df = pd.read_csv('genome_scores.csv')
imdb_data_df = pd.read_csv('imdb_data.csv')
movies_df = pd.read_csv('movies.csv')

# Merge dataframes on movieId
merged_df = pd.merge(train_df, genome_scores_df[:20000], on='movieId', how='left')
merged_df = pd.merge(merged_df, genome_tags_df, on='tagId', how='left')
merged_df = pd.merge(merged_df, imdb_data_df, on='movieId', how='left')
merged_df = pd.merge(merged_df, movies_df, on='movieId', how='left')

In [ ]:
print(merged_df.shape)
merged_df.head(1)

In [ ]:
import pandas as pd

def process_chunk(chunk):
    chunk['combined'] = (pd.Series(chunk[['userId', 'movieId', 'rating', 'timestamp', 'tagId', 'relevance', 'tag', 'title_cast', 'director', 'runtime', 'budget', 'plot_keywords', 'genres']]
                                   .fillna('')
                                   .values.tolist()).str.join(' '))
    return chunk

# Define the chunk size
chunk_size = 1000  # Adjust this number based on your memory capacity
num_chunks = len(merged_df) // chunk_size + 1

# Process the dataframe in chunks
chunks = [process_chunk(merged_df.iloc[i*chunk_size:(i+1)*chunk_size]) for i in range(num_chunks)]

# Concatenate all chunks back into a single dataframe
merged_df = pd.concat(chunks)

# Create titles and indices
titles = merged_df['title']
indices = pd.Series(merged_df.index, index=merged_df['title'])


In [ ]:
train_df.head()

In [ ]:
genome_tags_df.head()

In [ ]:
imdb_data_df.head()

In [ ]:
genome_scores_df.head()

In [ ]:
test_df.head()

In [ ]:
plt.hist(genome_scores_df['relevance'])
plt.show()

In [ ]:
plt.hist(train_df['rating'])
plt.show()

In [ ]:
movies_df.head()

In [ ]:
movies_df.shape